$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Les filtres convolutifs (partie 2)
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
import torch
from torch import nn
torch.__version__ # Ce notebook a été conçu avec la version '1.2.0' de pytorch

## L'ensemble CIFAR

Nous vous fournissons un sous-ensemble du jeu de données CIFAR10. Le jeu de donnée original provient de : https://www.cs.toronto.edu/~kriz/cifar.html

Il s’agit d’un problème de classification multi-classes; le jeu de données contient des images couleurs de taille
32 × 32 pixels représentant 10 catégories d’objets. Pour simplifier le problème et réduire le temps d’ap-
prentissage requis, nous vous suggérons de conserver seulement les trois premières catégories : «avion»,
«automobile» et «oiseau». Le répertoire `data/cifar` contient un fichier compressé par catégorie, chacun regroupant les images en format *PNG*.

La méthode `charger_cifar` du fichier `cifar_utils` permet d’extraire les images compressées du jeu de données et de les transformer en vecteur de 3 × 32 × 32 = 3072 nombres réels compris entre 0.0 et 1.0, qui sont la concaténation des valeurs des canaux rouge, vert et bleu.

In [ ]:
from cifar_utils import charger_cifar, afficher_grille_cifar

In [ ]:
repertoire_cifar = '../data/cifar/'
classes_cifar = [0, 1, 2]

data_x, data_y = charger_cifar(repertoire_cifar, classes_cifar)

In [ ]:
np.shape(data_x)

In [ ]:
data_y

Séparons les images en un ensemble d'apprentissage et un ensemble de test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.5, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

La méthode `afficher_grille_cifar` du fichier `cifar_utils` permet visualiser un ensemble d'images.

In [ ]:
indices_aleatoires = np.random.randint(len(train_y), size=40)
afficher_grille_cifar(train_x[indices_aleatoires])

## Apprentissage d'un réseau pleinement connecté

In [ ]:
from reseau_classif_generique import ReseauClassifGenerique

In [ ]:
nb_entrees = 3 * 32 * 32
nb_sorties = 3
nb_neurones_cachees = 50

archi_pleinement_connectee = nn.Sequential(
            nn.Linear(nb_entrees, nb_neurones_cachees),
            nn.ReLU(),
            nn.Linear(nb_neurones_cachees, nb_sorties),
            nn.LogSoftmax(dim=1)
        )

reseau_pc = ReseauClassifGenerique(archi_pleinement_connectee, eta=0.01, alpha=0.1, nb_epoques=500, 
                                   taille_batch=32, fraction_validation=.1, patience=20)

reseau_pc.apprentissage(train_x, train_y)

In [ ]:
train_pred = reseau_pc.prediction(train_x)
test_pred = reseau_pc.prediction(test_x)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

-----------
## Exercice

En vous inspirant du code réalisé lors de la première partie de ce TD, créer un réseau de neurones convolutifs pour l'ensemble CIFAR. La différence ici est que les filtres de convolutions devront prendre en considération les trois canaux de couleurs des images.

Expérimentez des solutions avec une ou deux couches de convolutions, ainsi qu'avec des quantités et des tailles de filtres différents.